## Question 2 EM

Yilin Gao (yg95)

In [2]:
import numpy as np

### simulate data

m = 6, n = 100,

for each m, 50% chance using machine 1, and 50% change using machine 2

theta1 = 0.8, theta2 = 0.3

1: broken, 0: not broken

In [3]:
X = np.zeros([6, 1])
for i in range(6):
    machine = np.random.rand() # random float in [0, 1) with uniform distribution
    broken_prop = 0
    if machine > 0.5: # machine 2
        broken_prop = 0.3
    else: # machine 1
        broken_prop = 0.8
    row = np.random.rand(1, 100) # random float in [0, 1) with uniform distribution
    broken_index = row < broken_prop
    X[i, 0] = np.sum(broken_index)

In [4]:
print(X.shape)
print(X)

(6, 1)
[[83.]
 [82.]
 [84.]
 [30.]
 [35.]
 [79.]]


### initialize parameters

In [5]:
w = np.ones([6, 2])
w = w / 2
theta = np.asarray([0.9, 0.1])

### run EM algorithm

In [6]:
def has_converged(a, b):
    '''
    tell if 2 numpy arrays are element-wise equal within a tolerance
    '''
    if (a is None or a is None):
        return False
    assert a.shape[0] == b.shape[0]
    # Returns True if two arrays are element-wise equal within a tolerance.
    return np.allclose(a, b)

In [7]:
def EM_onestep(X, n, w, theta):
    m = X.shape[0]
    assert w.shape[1] == theta.shape[0]
    k = w.shape[1]
    ## E step
    gamma = np.zeros([m, k])
    for i in range(k):
        gamma[:, i] = ((theta[i] ** X) * ((1 - theta[i]) ** (n - X))).flatten() * w[:, i]
    gamma_denominator = np.sum(gamma, axis=1).reshape((m, 1))
    gamma = gamma / gamma_denominator
    ## M step
    theta_new = np.zeros(k)
    w_new = np.zeros([m, k])
    for i in range(k):
        theta_denominator = np.sum(gamma[:, i])
        theta_nominator = np.dot(gamma[:, i], X) / n
        theta_new[i] = theta_nominator / theta_denominator
        w_new[:, i] = gamma[:, i]
    w_denominator = np.sum(w_new, axis=1).reshape((m, 1))
    w_new = w_new / w_denominator
    return w_new, theta_new

In [8]:
print(EM_onestep(X, 100, w, theta))

(array([[1.00000000e+00, 1.04711499e-63],
       [1.00000000e+00, 8.48163142e-62],
       [1.00000000e+00, 1.29273456e-65],
       [6.76549570e-39, 1.00000000e+00],
       [2.35898249e-29, 1.00000000e+00],
       [1.00000000e+00, 4.50748669e-56]]), array([0.82 , 0.325]))


In [9]:
def EM(X, n, w, theta):
    w_old, theta_old = None, None
    w_new, theta_new = w, theta
    while (not has_converged(w_old, w_new) or not has_converged(theta_old, theta_new)):
        w_old, theta_old = w_new, theta_new
        w_new, theta_new = EM_onestep(X, n, w_old, theta_old)
    return w_new, theta_new

In [10]:
print(EM(X, 100, w, theta))

(array([[1.00000000e+00, 2.61996290e-87],
       [1.00000000e+00, 2.00789926e-84],
       [1.00000000e+00, 3.41860059e-90],
       [5.08189425e-67, 1.00000000e+00],
       [1.34356757e-52, 1.00000000e+00],
       [1.00000000e+00, 9.03820638e-76]]), array([0.82 , 0.325]))
